In [ ]:
import mysql.connector


db = mysql.connector.connect(
    host="localhost,
    user="root",
    password="your_password",
    database="PS5 "
)

cursor = db.cursor()

# Function to place a new order
def place_order(customer_id, product_id, quantity):
    try:
        # Check if the product is available
        cursor.execute("SELECT quantity_available, price FROM Products WHERE product_id = %s", (product_id,))
        product_data = cursor.fetchone()

        if product_data and product_data[0] >= quantity:
            # Calculate the total cost
            total_cost = product_data[1] * quantity

            # Insert the order into the database
            cursor.execute("INSERT INTO Orders (customer_id, total_cost) VALUES (%s, %s)", (customer_id, total_cost))
            order_id = cursor.lastrowid

            # Insert order details
            cursor.execute("INSERT INTO OrderDetails (order_id, product_id, quantity_ordered) VALUES (%s, %s, %s)",
                           (order_id, product_id, quantity))

            # Update product quantity
            new_quantity = product_data[0] - quantity
            cursor.execute("UPDATE Products SET quantity_available = %s WHERE product_id = %s", (new_quantity, product_id))

            db.commit()
            return f"Order placed successfully. Order ID: {order_id}"
        else:
            return "Product is unavailable or quantity is insufficient."
    except Exception as e:
        db.rollback()
        return f"Error: {str(e)}"
    finally:
        db.close()

# Example usage:
result = place_order(1, 101, 2)
print(result)


In [ ]:
import csv

def generate_order_report():
    try:
        cursor.execute("SELECT Orders.order_id, Customers.name, Orders.order_date, Orders.total_cost "
                       "FROM Orders "
                       "INNER JOIN Customers ON Orders.customer_id = Customers.customer_id")

        orders = cursor.fetchall()

        with open('order_report.csv', 'w', newline='') as csvfile:
            fieldnames = ['Order ID', 'Customer Name', 'Order Date', 'Total Cost']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()

            for order in orders:
                writer.writerow({'Order ID': order[0], 'Customer Name': order[1], 'Order Date': order[2], 'Total Cost': order[3]})

        return "Order report generated successfully."
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage:
report_result = generate_order_report()
print(report_result)
